In [2]:
!pip install -q pogema==1.2.2
!pip install -q stable_baselines3==2.1.0
!pip install -q sb3-contrib==2.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 84.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━

In [3]:
import gymnasium as gym
from pogema import GridConfig
import pogema
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


GRID_LEN = 18

def custom_step(self, action: list):
        assert len(action) == self.grid_config.num_agents
        rewards = []

        terminated = []

        self.move_agents(action)
        self.update_was_on_goal()

        for agent_idx in range(self.grid_config.num_agents):

            c_x, c_y = self.grid.positions_xy[agent_idx]
            f_x, f_y = self.grid.finishes_xy[agent_idx]

            #d = math.sqrt((c_x - f_x) ** 2 + (c_y - f_y) ** 2)
            #reward = 1 - (d / (math.sqrt(2) * GRID_LEN))
            reward =  - ( (abs(c_x - f_x) + abs(c_y - f_y)) / (2 * GRID_LEN) )
            #print(f"[CURR] {c_x}, {c_y} [FINISH] {f_x}, {f_y} [DIST] {d} [REWARD] {reward}")


            on_goal = self.grid.on_goal(agent_idx)
            if on_goal and self.grid.is_active[agent_idx]:
                print("FINISH", reward)
                rewards.append(100)
            else:
                rewards.append(reward)
            terminated.append(on_goal)

        for agent_idx in range(self.grid_config.num_agents):
            if self.grid.on_goal(agent_idx):
                self.grid.hide_agent(agent_idx)
                self.grid.is_active[agent_idx] = False

        infos = self._get_infos()

        observations = self._obs()
        truncated = [False] * self.grid_config.num_agents
        return observations, rewards, terminated, truncated, infos

pogema.envs.Pogema.step = custom_step

grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    max_episode_steps=64,
    obs_radius=5
)

env = gym.make("Pogema-v0",grid_config=grid_config)


recurrent_ppo = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,

)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you 

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
!pip install -q optuna
!pip install -q optuna.pruners
!pip install -q optuna.samplers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement optuna.pruners (from versions: none)
ERROR: No matching distribution found for optuna.pruners
ERROR: Could not find a version that satisfies the requirement optuna.samplers (from versions: none)
ERROR: No matching distribution found for optuna.samplers


In [ ]:
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.2e5)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


ENV_ID = "Pogema-v0"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpLstmPolicy",
    "verbose": 1,
    "env": ENV_ID
}

def sample_recurrentppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for RecurrentPPO hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # Display true values.
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("gae_lambda_", gae_lambda)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]} if net_arch == "tiny" else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]


    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }

class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_recurrentppo_params(trial))
    # Create the RL model.
    model = RecurrentPPO(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

[I 2023-11-11 06:19:16,249] A new study created in memory with name: no-name-aa6382d5-0800-4d07-808c-7613ef48c482


流式输出内容被截断，只能显示最后 5000 行内容。
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 54.2        |
|    ep_rew_mean          | 15.4        |
| time/                   |             |
|    fps                  | 20          |
|    iterations           | 3515        |
|    time_elapsed         | 5561        |
|    total_timesteps      | 112480      |
| train/                  |             |
|    approx_kl            | 0.005578246 |
|    clip_fraction        | 0.0125      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.2        |
|    explained_variance   | -0.000551   |
|    learning_rate        | 0.000223    |
|    loss                 | -0.0378     |
|    n_updates            | 35140       |
|    policy_gradient_loss | -0.0147     |
|    value_loss           | 0.0643      |
-----------------------------------------
-----------------------------------------
| roll

[I 2023-11-11 07:58:06,729] Trial 0 finished with value: -8.574074333333334 and parameters: {'gamma': 0.00012576767542601166, 'max_grad_norm': 1.6300062831477742, 'gae_lambda': 0.07988138019584923, 'exponent_n_steps': 5, 'lr': 0.00022323802341970723, 'ent_coef': 0.01662132875986711, 'ortho_init': True, 'net_arch': 'small', 'activation_fn': 'tanh'}. Best is trial 0 with value: -8.574074333333334.


Number of finished trials:  1
Best trial:
  Value:  -8.574074333333334
  Params: 
    gamma: 0.00012576767542601166
    max_grad_norm: 1.6300062831477742
    gae_lambda: 0.07988138019584923
    exponent_n_steps: 5
    lr: 0.00022323802341970723
    ent_coef: 0.01662132875986711
    ortho_init: True
    net_arch: small
    activation_fn: tanh
  User attrs:
    gamma_: 0.999874232324574
    gae_lambda_: 0.9201186198041508
    n_steps: 32


In [5]:

val_env = gym.make("Pogema-v0",grid_config=grid_config)

val_recurrent_ppo = RecurrentPPO(
     "MlpLstmPolicy",
    val_env,
    verbose=1,
    gamma=0.999874232324574,
    learning_rate=0.00022323802341970723,



)

mean_reward, std_reward = evaluate_policy(
    val_recurrent_ppo,
    val_recurrent_ppo.get_env(),
    deterministic=True,
    n_eval_episodes=20,
)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
FINISH -0.0
mean_reward: -6.66 +/- 24.94


In [6]:
# Train agent and save it
recurrent_ppo.learn(total_timesteps=int(1.2e5))
recurrent_ppo.save("saved/recurrentppo_baseline")

流式输出内容被截断，只能显示最后 5000 行内容。
| train/                  |              |
|    approx_kl            | 0.0049386392 |
|    clip_fraction        | 0.0109       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.74        |
|    explained_variance   | 0.00661      |
|    learning_rate        | 0.0003       |
|    loss                 | 320          |
|    n_updates            | 7010         |
|    policy_gradient_loss | -0.00154     |
|    value_loss           | 647          |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 55.4        |
|    ep_rew_mean          | 8.33        |
| time/                   |             |
|    fps                  | 21          |
|    iterations           | 703         |
|    time_elapsed         | 4108        |
|    total_timesteps      | 89984       |
| train/                  |             |
|    approx_kl            | 0.0146182

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'saved' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [8]:
# RANDOM SEED
from IPython.display import SVG, display
from pogema.animation import AnimationMonitor, AnimationConfig

env = AnimationMonitor(env)

def evaluate_success_rate(model, env, num_episodes=1000):
    success_count = 0
    step_array = []
    rewards_arr = []
    all_rewards_arr = []
    for i in range(num_episodes):
        print(f'---{i}---')
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 64
        steps_taken = 0
        reward_acc = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done, reward)
            reward_acc += reward
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                print("acc reward", reward_acc)
                step_array.append(steps_taken)
                rewards_arr.append(reward_acc)
                break

        all_rewards_arr.append(reward_acc)

    success_rate = success_count / num_episodes
    return success_rate, step_array, rewards_arr, all_rewards_arr

success_rate,step_array, reward_arr, all_rewards_arr = evaluate_success_rate(recurrent_ppo, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")
print(f"Reward Array : {reward_arr}")
print(f"All Reward Array : {all_rewards_arr}")

# Calculate the average steps to termination
average_steps = sum(step_array) / len(step_array)
print("Average steps to termination:", average_steps)

流式输出内容被截断，只能显示最后 5000 行内容。
3 40 84 False -0.16666666666666666
4 39 84 False -0.16666666666666666
4 38 84 False -0.16666666666666666
3 37 84 False -0.16666666666666666
3 36 84 False -0.16666666666666666
4 35 84 False -0.16666666666666666
3 34 84 False -0.16666666666666666
4 33 84 False -0.16666666666666666
0 32 84 False -0.16666666666666666
3 31 84 False -0.16666666666666666
3 30 84 False -0.16666666666666666
0 29 84 False -0.16666666666666666
0 28 84 False -0.16666666666666666
4 27 84 False -0.16666666666666666
3 26 84 False -0.16666666666666666
3 25 84 False -0.16666666666666666
4 24 84 False -0.16666666666666666
3 23 84 False -0.16666666666666666
3 22 84 False -0.16666666666666666
3 21 84 False -0.16666666666666666
3 20 84 False -0.16666666666666666
4 19 84 False -0.16666666666666666
3 18 84 False -0.16666666666666666
4 17 84 False -0.16666666666666666
3 16 84 False -0.16666666666666666
3 15 84 False -0.16666666666666666
3 14 84 False -0.16666666666666666
4 13 84 False -0.16666666666

In [9]:
all_rewards_arr

[-4.972222222222221,
 -8.27777777777778,
 -8.888888888888902,
 -8.916666666666673,
 -3.611111111111107,
 -16.388888888888896,
 -4.861111111111109,
 -9.666666666666666,
 -7.666666666666659,
 99.91666666666667,
 -9.666666666666675,
 -13.083333333333343,
 -3.5555555555555505,
 -4.916666666666665,
 -11.97222222222223,
 -10.666666666666663,
 -7.388888888888878,
 99.91666666666667,
 -6.722222222222216,
 -7.194444444444439,
 -2.0833333333333317,
 -9.111111111111118,
 -3.5555555555555505,
 -4.111111111111104,
 -4.0833333333333295,
 -13.388888888888893,
 -5.333333333333331,
 -5.333333333333331,
 -3.0277777777777737,
 -7.083333333333325,
 -6.666666666666658,
 95.80555555555556,
 -10.138888888888898,
 99.72222222222223,
 98.58333333333333,
 -8.277777777777771,
 -11.250000000000002,
 -7.472222222222212,
 -12.861111111111116,
 -12.388888888888898,
 -13.444444444444443,
 99.83333333333333,
 -8.111111111111102,
 -4.499999999999997,
 -8.888888888888902,
 -14.222222222222202,
 -9.27777777777779,
 -9.63

In [10]:
step_array

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[4,
 3,
 62,
 7,
 29,
 7,
 12,
 1,
 11,
 17,
 1,
 7,
 17,
 4,
 25,
 5,
 16,
 1,
 9,
 2,
 1,
 44,
 1,
 16,
 47,
 53,
 2,
 4,
 2,
 5,
 17,
 1,
 1,
 2,
 1,
 2,
 1,
 3,
 20,
 1,
 1,
 17,
 5,
 15,
 2,
 10,
 3,
 42,
 34,
 38,
 1,
 29,
 2,
 4,
 18,
 14,
 17,
 2,
 3,
 20,
 54,
 49,
 3,
 1,
 7,
 5,
 31,
 1,
 22,
 10,
 1,
 7,
 3,
 2,
 1,
 59,
 1,
 6,
 1,
 1,
 45,
 1,
 2,
 15,
 2,
 58,
 1,
 3,
 1]

In [11]:
reward_arr

[99.91666666666667,
 99.91666666666667,
 95.80555555555556,
 99.72222222222223,
 98.58333333333333,
 99.83333333333333,
 99.52777777777777,
 100,
 99.69444444444444,
 99.27777777777777,
 100,
 99.69444444444444,
 98.83333333333333,
 99.88888888888889,
 99.33333333333333,
 99.80555555555556,
 99.58333333333333,
 100,
 99.63888888888889,
 99.97222222222223,
 100,
 96.88888888888889,
 100,
 99.55555555555556,
 97.88888888888889,
 96.36111111111111,
 99.97222222222223,
 99.88888888888889,
 99.97222222222223,
 99.83333333333333,
 99.55555555555556,
 100,
 100,
 99.97222222222223,
 100,
 99.97222222222223,
 100,
 99.94444444444444,
 99.33333333333333,
 100,
 100,
 99.25,
 99.80555555555556,
 98.97222222222223,
 99.97222222222223,
 99.38888888888889,
 99.91666666666667,
 97.77777777777777,
 98.30555555555556,
 98.02777777777779,
 100,
 99.05555555555556,
 99.97222222222223,
 99.88888888888889,
 99.52777777777777,
 99.36111111111111,
 99.41666666666667,
 99.97222222222223,
 99.94444444444444,
